In [2]:
# import matplotlib.pyplot as plt
import numpy as np
# import pandas as pd
# import tensorflow as tf
# import sklearn
import cv2
# from PIL import Image
import math
# import scipy
# from scipy import stats
# %matplotlib notebook

In [73]:
class Code_rec():
    # 用cv2转成HSV颜色空间，轻松提取不同色调的字符str
    def extract_image(self, four_str_color, im_hsv):
        image_list = []
        for i in four_str_color:
            lower = np.array([i, 0, 0])
            upper = np.array([i, 255, 255])
            mask = cv2.inRange(im_hsv,lowerb=lower, upperb=upper)
            image_list.append(mask)
        return image_list

    #用递归计算叠加最大值（其实有点小题大做了）——封装好了，以后只管调用就好，懒得再看了，也不写备注了
    def calculate_index(self, lst):
        ll = []
        def foo(lst, i):
            try:
                if lst[i] == 0:
                    return 0
                else:
                    x = lst[i] + foo(lst, i+1)
                    return x
            except:
                return lst[i]
        for i in range(len(lst)):
            ll.append(foo(lst, i))
        #最大的索引，也就是字符出现的最左边的索引位置
        left_index = np.argmax(ll)
        for n, i in enumerate(ll[left_index:]):
            if i == 0:
                length = n
                break
        mid_index = left_index + length // 2
        #返回每张img的中间位置索引
        return mid_index
    
         
    def get_img_order_index(self, imgs):
        """
        得到提取出来的4个不同颜色的img列表，计算他们的排列顺序、中心位置，并返回
        """
        count = 0
        wrapper_dict = {}
        for num, img in enumerate(imgs):
            pixel_num_lst = []
            for index, col in enumerate(img.T):
                pixel_num = 0
                for row in col:
                    count += 1
                    if row == 255:
                        pixel_num += 1
                pixel_num_lst.append(pixel_num)
            
            mid_index = self.calculate_index(pixel_num_lst)  # 返回字符的左右边界的索引
            wrapper_dict[num] = mid_index
#         print(wrapper_dict)
        order_index = np.argsort([mid for mid in wrapper_dict.values()]) # 得到从左到右的排序的图片索引
        ordered_imgs = [j for j in  zip(np.array(imgs)[order_index], [wrapper_dict[i] for i in order_index])]
        #上面ordered_imgs 的返回结构：[(<im1>, mid_index), (<im2>, mid_index)...]
#         print(ordered_imgs)
        return ordered_imgs

    def cut_image(self, imgs):
        """
        按中间位置左右切割20个像素，得到一个小正方形的str二值图案
        """
        img_cuted_lst = []
        for i in imgs:
            img_cuted = i[0][:, i[1]-20:i[1]+20]
#             print(img_cuted)
            img_cuted_lst.append(img_cuted)
        return img_cuted_lst
        
    def save_image(self, imgs, filename=None):
        num = 1
        if not filename:
            for img in imgs:
                cv2.imwrite("./code_imgs/img%s.jpg"%num, img)
                num += 1
        else:
            for img in imgs:
                cv2.imwrite("./code_imgs/img%s--%s.jpg"%(filename,num), img)
                num += 1
        
    def convert_image(self, filename):
        """
        输入：原始的验证码的文件路径名
        输出：分割好的 4 张二值化的小正方形的img矩阵列表  
            （format: [im1, im2, im3, im4] ）
             (use: 1.存储 2.训练模型 3.喂给模型得到预测结果 )
        
        """
        im = cv2.imread(filename)
        cv2.imshow("im-", im)
        im = cv2.morphologyEx(im, cv2.MORPH_CLOSE, (2, 1))
        im = cv2.medianBlur(im, 3)

        im_hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)#把BGR图像转换为HSV格式
        im_bins = np.bincount(im_hsv.reshape(-1, 3)[:, 0])
        four_str_color = np.argsort(im_bins)[-1::-1][1:5]

        image_extract_list = self.extract_image(four_str_color, im_hsv)
        img_order_lst = self.get_img_order_index(image_extract_list)  # 得到img的从左到右的排列顺序，以及每个img的left_right索引，等待切割
        img_converted_lst = self.cut_image(img_order_lst)
#         self.save_image(img_cuted_lst)
        
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        return img_converted_lst
    
# code_rec = Code_rec()
# img_converted_lst = code_rec.convert_image("t2.jpg")
# code_rec.save_image(img_converted_lst)

In [74]:
code_rec = Code_rec()
for i in range(1, 7):
    imgs = code_rec.convert_image("t%s.jpg"%i)
    code_rec.save_image(imgs, "%s"%i)

UnboundLocalError: local variable 'length' referenced before assignment

In [72]:
for i in range(1, 1000):
    im = cv2.imread("./code_img/t%s.jpg"%i)
    im = im[:40, :120, :]
#     print(im.shape)
    cv2.imwrite("./code_img/t%s.jpg"%i, im)

In [ ]:
ss.head(20)

In [ ]:
ss.tshift(-7) / ss

In [ ]:
np.arange(48, 60).sum()/12

In [ ]:
s.asfreq("BA")